# CS-2376 Apriori Algorithm
### Team Members
1. Jenish Raj Bajracharya
2. Nidup Dorji
3. Satpreet Makhija


In [1]:
import pandas as pd
import numpy as np
import operator
from itertools import combinations

## Data Pre-processing
1. Reading Data
2. Getting unique dataset

In [2]:
dataframe = pd.read_csv("dataset/tesco2.csv",  header=None)
dataframe.head()
dataframe = dataframe.values

data = set()
for items in dataframe:
    for sub_item in items:
        data.add(sub_item)
# unique_data = list(data)
# print(unique_data)



In [6]:

def get_frequent_set(min_support, dataframe,data):
    temp_data = dict()
    itemset = data
    for i in itemset:
        count = 0
        for d in dataframe:
            if i in d:
                count += 1
        temp_data[i] = count
        if temp_data[i] < min_support:
            temp_data.pop(i)

    return sorted(temp_data.keys(), key=operator.itemgetter(0))


# def get_item_combination():
A1 = get_frequent_set(2, dataframe,data)
powerset = list(combinations(A1,2))
print (powerset)    


[('apple', 'beer'), ('apple', 'chicken'), ('apple', 'google'), ('apple', 'milk'), ('apple', 'mango'), ('apple', 'rice'), ('beer', 'chicken'), ('beer', 'google'), ('beer', 'milk'), ('beer', 'mango'), ('beer', 'rice'), ('chicken', 'google'), ('chicken', 'milk'), ('chicken', 'mango'), ('chicken', 'rice'), ('google', 'milk'), ('google', 'mango'), ('google', 'rice'), ('milk', 'mango'), ('milk', 'rice'), ('mango', 'rice')]
